In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# -----------------------------
# Domain & Material Parameters
# -----------------------------
L, W = 1.0, 0.5
lambda_ = 5.0e9
mu = 5.0e9
h  = 1.0

# -----------------------------
# PINN Model (same shape function)
# -----------------------------
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 2)
        )

    def forward(self, x, y):
        # x, y: shape (N,1)
        inputs = torch.cat([x, y], dim=1)  # shape (N,2)
        raw = self.net(inputs)             # shape (N,2)

        enforced_out = torch.zeros_like(raw)
        enforced_out[:, 0] = (x.flatten() + 0.5) * raw[:, 0]  # u_x
        enforced_out[:, 1] = (x.flatten() + 0.5) * raw[:, 1]  # u_y

        # Example corner constraint at x=0.5
        corner_mask = (x[:, 0] == 0.5)
        enforced_out[corner_mask, 1] = 0.0

        return enforced_out

# -----------------------------
# 1) Strain & Stress
# -----------------------------
def strain_tensor(u_x, u_y, x, y):
    u_x_x = torch.autograd.grad(u_x, x,
                                grad_outputs=torch.ones_like(u_x),
                                retain_graph=True,
                                create_graph=True)[0]
    u_y_y = torch.autograd.grad(u_y, y,
                                grad_outputs=torch.ones_like(u_y),
                                retain_graph=True,
                                create_graph=True)[0]
    u_x_y = torch.autograd.grad(u_x, y,
                                grad_outputs=torch.ones_like(u_x),
                                retain_graph=True,
                                create_graph=True)[0]
    u_y_x = torch.autograd.grad(u_y, x,
                                grad_outputs=torch.ones_like(u_y),
                                retain_graph=True,
                                create_graph=True)[0]

    E_xx = u_x_x
    E_yy = u_y_y
    E_xy = 0.5 * (u_x_y + u_y_x)
    return E_xx, E_yy, E_xy

def stress_tensor(E_xx, E_yy, E_xy):
    scale_factor = 1e9
    trace_E = E_xx + E_yy
    sigma_xx = h * ((lambda_/scale_factor)*trace_E + 2*(mu/scale_factor)*E_xx)
    sigma_yy = h * ((lambda_/scale_factor)*trace_E + 2*(mu/scale_factor)*E_yy)
    sigma_xy = h * (2*(mu/scale_factor)*E_xy)
    return sigma_xx, sigma_yy, sigma_xy

# -----------------------------
# 2) PDE Residual (Interior)
# -----------------------------
def pde_loss_vectorized(model, x, y):
    """
    Return PDE residual^2 per point (shape (N,)).
    PDE: sigma_xx_x + sigma_xy_y = 0, sigma_yy_y + sigma_xy_x = 0
    """
    x_ = x.clone().detach().requires_grad_(True)
    y_ = y.clone().detach().requires_grad_(True)

    u = model(x_, y_)
    u_x = u[:, 0:1]
    u_y = u[:, 1:2]

    E_xx, E_yy, E_xy = strain_tensor(u_x, u_y, x_, y_)
    sigma_xx, sigma_yy, sigma_xy = stress_tensor(E_xx, E_yy, E_xy)

    sigma_xx_x = torch.autograd.grad(sigma_xx, x_,
                                     grad_outputs=torch.ones_like(sigma_xx),
                                     retain_graph=True,
                                     create_graph=True)[0]
    sigma_xy_y = torch.autograd.grad(sigma_xy, y_,
                                     grad_outputs=torch.ones_like(sigma_xy),
                                     retain_graph=True,
                                     create_graph=True)[0]
    sigma_yy_y = torch.autograd.grad(sigma_yy, y_,
                                     grad_outputs=torch.ones_like(sigma_yy),
                                     retain_graph=True,
                                     create_graph=True)[0]
    sigma_xy_x = torch.autograd.grad(sigma_xy, x_,
                                     grad_outputs=torch.ones_like(sigma_xy),
                                     retain_graph=True,
                                     create_graph=True)[0]

    residual_x = sigma_xx_x + sigma_xy_y
    residual_y = sigma_yy_y + sigma_xy_x

    return (residual_x**2 + residual_y**2).flatten()

# -----------------------------
# 3) Boundary Losses (A,D,B,C)
# -----------------------------
def boundary_A_loss(model, x, y):
    """
    A: x=-L/2 => (u_x=0, u_y=0)
    => sum of squares of [u_x,u_y]
    """
    u = model(x, y)
    return torch.sum(u**2, dim=1)

def boundary_D_loss(model, x, y, L):
    """
    D: x=+L/2 => (u_x=0.025L, u_y=0)
    => (u_x - 0.025L)^2 + (u_y)^2
    """
    u = model(x, y)
    diff_x = u[:,0] - (0.025*L)
    diff_y = u[:,1]
    return diff_x**2 + diff_y**2

def boundary_B_loss(model, x, y):
    """
    B: y=+W/2 => traction-free => sigma_yy=0, sigma_xy=0
    => (sigma_yy^2 + sigma_xy^2)
    """
    x_ = x.clone().detach().requires_grad_(True)
    y_ = y.clone().detach().requires_grad_(True)
    u = model(x_, y_)
    u_x, u_y = u[:,0:1], u[:,1:2]

    E_xx_B, E_yy_B, E_xy_B = strain_tensor(u_x, u_y, x_, y_)
    sigma_xx_B, sigma_yy_B, sigma_xy_B = stress_tensor(E_xx_B, E_yy_B, E_xy_B)
    return (sigma_yy_B**2 + sigma_xy_B**2).flatten()

def boundary_C_loss(model, x, y):
    """
    C: y=-W/2 => traction-free => sigma_yy=0, sigma_xy=0
    """
    x_ = x.clone().detach().requires_grad_(True)
    y_ = y.clone().detach().requires_grad_(True)
    u = model(x_, y_)
    u_x, u_y = u[:,0:1], u[:,1:2]

    E_xx_C, E_yy_C, E_xy_C = strain_tensor(u_x, u_y, x_, y_)
    sigma_xx_C, sigma_yy_C, sigma_xy_C = stress_tensor(E_xx_C, E_yy_C, E_xy_C)
    return (sigma_yy_C**2 + sigma_xy_C**2).flatten()

# -----------------------------
# 4) Corner Constraint
# -----------------------------
def corner_loss(model, x_corner, y_corner, weight=1e9):
    """
    Enforce (u_x=0,u_y=0) at corner (x=-L/2,y=-W/2) with large weight
    """
    u_c = model(x_corner, y_corner)  # shape (1,2)
    return weight * torch.sum(u_c**2, dim=1)  # shape (1,)

# -----------------------------
# 5) Combined vectorized loss that returns
#    (total_loss, sub_losses_dict)
#    so you can print each sub-loss
# -----------------------------
def combined_loss_vectorized(model, x_all, y_all, L, W, corner_weight=1e9, tol=1e-6):
    """
    1) PDE for interior
    2) Boundaries:
       A => x=-L/2, D => x=+L/2
       B => y=+W/2 (traction-free)
       C => y=-W/2 (traction-free)
    3) Corner => (x=-L/2,y=-W/2) with large weight
    We build a sub-loss dict: {pde, A, D, B, C, corner}.
    Then total_loss = mean(...) + corner_loss
    """
    # sub-loss dictionary to store mean of each portion
    sub_losses = {
        'pde': 0.0,
        'A':   0.0,
        'D':   0.0,
        'B':   0.0,
        'C':   0.0,
        'corner': 0.0
    }

    xf = x_all.flatten()
    yf = y_all.flatten()

    maskA = torch.isclose(xf, torch.tensor(-L/2, device=xf.device), atol=tol)
    maskD = torch.isclose(xf, torch.tensor(+L/2, device=xf.device), atol=tol)
    maskB = torch.isclose(yf, torch.tensor(+W/2, device=yf.device), atol=tol)
    maskC = torch.isclose(yf, torch.tensor(-W/2, device=yf.device), atol=tol)

    mask_pde = ~(maskA | maskD | maskB | maskC)

    N = x_all.shape[0]
    pointwise_loss = torch.zeros(N, dtype=torch.float32, device=x_all.device)

    # PDE
    if mask_pde.any():
        xp = x_all[mask_pde].view(-1,1)
        yp = y_all[mask_pde].view(-1,1)
        pde_vals = pde_loss_vectorized(model, xp, yp)
        pointwise_loss[mask_pde] = pde_vals
        sub_losses['pde'] = float(torch.mean(pde_vals))

    # Boundary A
    if maskA.any():
        xA = x_all[maskA].view(-1,1)
        yA = y_all[maskA].view(-1,1)
        valsA = boundary_A_loss(model, xA, yA)
        pointwise_loss[maskA] = valsA
        sub_losses['A'] = float(torch.mean(valsA))

    # Boundary D
    if maskD.any():
        xD = x_all[maskD].view(-1,1)
        yD = y_all[maskD].view(-1,1)
        valsD = boundary_D_loss(model, xD, yD, L)
        pointwise_loss[maskD] = valsD
        sub_losses['D'] = float(torch.mean(valsD))

    # Boundary B => traction-free top
    if maskB.any():
        xB = x_all[maskB].view(-1,1)
        yB = y_all[maskB].view(-1,1)
        valsB = boundary_B_loss(model, xB, yB) / 1e18  # scale
        pointwise_loss[maskB] = valsB
        sub_losses['B'] = float(torch.mean(valsB))

    # Boundary C => traction-free bottom
    if maskC.any():
        xC = x_all[maskC].view(-1,1)
        yC = y_all[maskC].view(-1,1)
        valsC = boundary_C_loss(model, xC, yC) / 1e18  # scale
        pointwise_loss[maskC] = valsC
        sub_losses['C'] = float(torch.mean(valsC))

    # Now corner constraint => single point
    x_corner = torch.tensor([[-L/2]], dtype=torch.float32, requires_grad=True)
    y_corner = torch.tensor([[-W/2]], dtype=torch.float32, requires_grad=True)
    corner_vals = corner_loss(model, x_corner, y_corner, weight=corner_weight)
    sub_losses['corner'] = float(torch.mean(corner_vals))

    # Sum boundary & PDE => mean
    total_bc_pde = torch.mean(pointwise_loss)
    # Then add corner (not averaged with the rest, your snippet multiplied by big weight)
    total_loss = total_bc_pde + corner_vals.mean()

    return total_loss, sub_losses

# -----------------------------
# 6) Single training loop that prints sub-losses
# -----------------------------
def train_pinn_vectorized(model, optimizer, n_epochs, n_points, L, W):
    loss_history = []

    for epoch in range(n_epochs):
        # 1) Sample PDE points in full domain: x in [-L/2, L/2], y in [-W/2, W/2]
        x_in = (torch.rand((n_points,1)) * L) - (L/2)
        y_in = (torch.rand((n_points,1)) * W) - (W/2)

        # 2) Sample boundary lines
        #  A => x=-L/2
        yA = torch.linspace(-W/2, W/2, 100).reshape(-1,1)
        xA = -L/2 * torch.ones_like(yA)

        #  D => x=+L/2
        yD = torch.linspace(-W/2, W/2, 100).reshape(-1,1)
        xD = +L/2 * torch.ones_like(yD)

        #  B => y=+W/2
        xB = torch.linspace(-L/2, L/2, 100).reshape(-1,1)
        yB = +W/2 * torch.ones_like(xB)

        #  C => y=-W/2
        xC = torch.linspace(-L/2, L/2, 100).reshape(-1,1)
        yC = -W/2 * torch.ones_like(xC)

        # 3) Combine PDE + boundaries
        all_x = torch.cat([x_in, xA, xD, xB, xC], dim=0)
        all_y = torch.cat([y_in, yA, yD, yB, yC], dim=0)

        # 4) Evaluate combined PDE+BC, plus corner
        total_loss, sub_dict = combined_loss_vectorized(model, all_x, all_y, L, W, corner_weight=1e9, tol=1e-6)

        # 5) Backprop
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        loss_history.append(total_loss.item())

        # 6) Print each sub-loss every 500 epochs (or as you wish)
        if epoch % 500 == 0:
            print(
                f"Epoch {epoch:5d} | total={total_loss.item():.4e} | "
                f"pde={sub_dict['pde']:.4e}, A={sub_dict['A']:.4e}, "
                f"D={sub_dict['D']:.4e}, B={sub_dict['B']:.4e}, "
                f"C={sub_dict['C']:.4e}, corner={sub_dict['corner']:.4e}"
            )

    return loss_history


# -----------------------------
# 7) Run training + Evaluate
# -----------------------------
model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 10000
n_points = 1000  # PDE interior points each epoch

loss_history = train_pinn_vectorized(model, optimizer, n_epochs, n_points, L, W)

# ~~~ Evaluate results ~~~
file_path = '/Users/murat/Downloads/data.csv'
comparison_data = pd.read_csv(file_path)

x_values = torch.tensor(comparison_data['X'].values, dtype=torch.float32).reshape(-1, 1)
y_values = torch.tensor(comparison_data['Y'].values, dtype=torch.float32).reshape(-1, 1)
u_x_actual = comparison_data['u_x_actual'].values
u_y_actual = comparison_data['u_y_actual'].values

model.eval()
with torch.no_grad():
    u_pred = model(x_values, y_values)

u_x_pred = u_pred[:,0].numpy()
u_y_pred = u_pred[:,1].numpy()

comparison_data['u_x_pred'] = u_x_pred
comparison_data['u_y_pred'] = u_y_pred

comparison_data['error_u_x'] = abs(u_x_actual - u_x_pred)
comparison_data['error_u_y'] = abs(u_y_actual - u_y_pred)

def safe_percent_error(a, p):
    if np.isclose(a, 0.0):
        return 0.0
    else:
        return abs(a - p) / abs(a) * 100.0

comparison_data['percent_error_u_x'] = [
    safe_percent_error(a, p) for a, p in zip(u_x_actual, u_x_pred)
]
comparison_data['percent_error_u_y'] = [
    safe_percent_error(a, p) for a, p in zip(u_y_actual, u_y_pred)
]

avg_percent_error_x = comparison_data['percent_error_u_x'].mean()
avg_percent_error_y = comparison_data['percent_error_u_y'].mean()

print(comparison_data)
print(f"Average Percent Error for u_x: {avg_percent_error_x:.2f}%")
print(f"Average Percent Error for u_y: {avg_percent_error_y:.2f}%")


Epoch     0 | total=1.2468e-01 | pde=1.7424e-01, A=0.0000e+00, D=3.2040e-03, B=4.3970e-20, C=2.7293e-20, corner=0.0000e+00
Epoch   500 | total=1.3035e-05 | pde=1.8240e-05, A=0.0000e+00, D=9.5092e-08, B=2.4853e-20, C=2.5152e-20, corner=0.0000e+00
Epoch  1000 | total=2.9595e-06 | pde=4.1315e-06, A=0.0000e+00, D=1.2966e-07, B=2.2694e-20, C=2.3223e-20, corner=0.0000e+00
Epoch  1500 | total=1.3946e-06 | pde=1.9415e-06, A=0.0000e+00, D=1.2018e-07, B=2.2267e-20, C=2.2864e-20, corner=0.0000e+00
Epoch  2000 | total=7.9037e-07 | pde=1.0969e-06, A=0.0000e+00, D=1.0579e-07, B=2.2245e-20, C=2.2932e-20, corner=0.0000e+00
Epoch  2500 | total=4.9407e-07 | pde=6.8323e-07, A=0.0000e+00, D=9.3044e-08, B=2.2459e-20, C=2.3396e-20, corner=0.0000e+00
Epoch  3000 | total=3.3084e-07 | pde=4.5658e-07, A=0.0000e+00, D=7.2502e-08, B=2.2511e-20, C=2.3396e-20, corner=0.0000e+00
Epoch  3500 | total=2.1609e-07 | pde=2.9701e-07, A=0.0000e+00, D=6.0555e-08, B=2.2211e-20, C=2.3166e-20, corner=0.0000e+00
Epoch  4000 | to